In [ ]:
# --- Filter sequences to exact SEQ_LEN ---
filtered_data = []
removed_counts = Counter()
for seq in data:
    if len(seq["landmarks"]) == SEQ_LEN:
        filtered_data.append(seq)
    else:
        removed_counts[seq["label"]] += 1

print("Sequences removed due to length != 10:", removed_counts)
print("Sequences kept per class:", Counter(seq['label'] for seq in filtered_data))

In [ ]:
import numpy as np
import random

def augment_fire_sequences(sequences, target_count, jitter=2, noise_std=0.01, frame_dropout_prob=0.1):
    """
    Augment short dynamic gesture sequences to increase dataset size.

    Args:
        sequences: List of dicts, each {"landmarks": [[...]], "label": "Fire"}.
        target_count: Desired total number of sequences after augmentation.
        jitter: Max frames to shift start/end.
        noise_std: Standard deviation of Gaussian noise added to landmarks.
        frame_dropout_prob: Probability to randomly drop a frame.

    Returns:
        Augmented list of sequences.
    """
    augmented = sequences.copy()
    while len(augmented) < target_count:
        seq = random.choice(sequences)
        landmarks = seq["landmarks"]
        seq_len = len(landmarks)

        # --- Temporal jittering ---
        start_shift = random.randint(0, min(jitter, seq_len-1))
        end_shift = random.randint(0, min(jitter, seq_len-1))
        new_seq = landmarks[start_shift: seq_len - end_shift]

        # --- Frame dropout / repetition ---
        frames = []
        for frame in new_seq:
            if random.random() < frame_dropout_prob and len(new_seq) > 1:
                continue  # drop this frame
            frames.append(frame)
        if len(frames) == 0:
            frames = new_seq.copy()

        # --- Add small Gaussian noise ---
        noisy_frames = []
        for frame in frames:
            frame_array = np.array(frame)
            frame_array += np.random.normal(0, noise_std, frame_array.shape)
            noisy_frames.append(frame_array.tolist())

        augmented.append({
            "landmarks": noisy_frames,
            "label": seq["label"]
        })

    return augmented

import json

# Load dynamic sequences
with open("dynamic_sequences_10.json", "r") as f:
    dynamic_sequences = json.load(f)
target_fire_count = 400
fire_sequences_augmented = augment_fire_sequences(dynamic_sequences, target_fire_count)
print(f"Fire sequences augmented: {len(fire_sequences_augmented)}")

with open("fire_sequences_augmented.json", "w") as f:
    json.dump(fire_sequences_augmented, f)

In [ ]:
import json
import random

# --- Load JSON files ---
with open("Old TCN Files/idle_sequences_10.json", "r") as f:
    idle_sequences = json.load(f)

with open("Old TCN Files/aim_sequences_10.json", "r") as f:
    aim_sequences = json.load(f)

with open("Old TCN Files/fire_sequences_10_filtered.json", "r") as f:
    fire_sequences = json.load(f)

# --- Sample 125 sequences from each ---
idle_sample = random.sample(idle_sequences, 74)  # all 125
aim_sample = random.sample(aim_sequences, 74)
fire_sample = fire_sequences

# --- Combine and shuffle ---
all_sequences = idle_sample + aim_sample + fire_sample
random.shuffle(all_sequences)

# --- Save combined JSON ---
with open("Old TCN Files/balanced_sequences_take2.json", "w") as f:
    json.dump(all_sequences, f)

In [ ]:
import json

# Load the static sequences
with open("Old TCN Files/static_sequences_10.json", "r") as f:
    static_sequences = json.load(f)

# Separate sequences by label
idle_sequences = [seq for seq in static_sequences if seq["label"] == "Idle"]
aim_sequences = [seq for seq in static_sequences if seq["label"] == "Aim"]

# Save them into separate files
with open("Old TCN Files/idle_sequences_10.json", "w") as f:
    json.dump(idle_sequences, f)

with open("Old TCN Files/aim_sequences_10.json", "w") as f:
    json.dump(aim_sequences, f)

print(f"Idle: {len(idle_sequences)} sequences")
print(f"Aim: {len(aim_sequences)} sequences")